In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
headers = {"Accept-Language": "en-US, en;q=0.5"}
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\yfird\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [24]:
page=1
Name=[]
Price=[]
while page !=15:
    url="https://ounje.ng/shop/page/{}/".format(page)
    driver.get(url)
    results = driver.page_source
    soup = BeautifulSoup(results,"lxml")

    products=soup.find_all('div', class_="product-info")
    for item in products:
        name=item.find('h3', class_="product-name product_title").a.text
        Name.append(name)
        #print(Name)
        #price
        price=item.find('bdi')
        if price is not None:
             price= price.text.strip()      
        else:
             price = 'NaN'
        Price.append(price)
        #print(Price)
        
        
    page=page+1

In [25]:
len(Price)

321

In [92]:
items=pd.DataFrame({
    'Name':Name,
    'Price':Price
})

In [93]:
items.head()

,Name,Price
0,Groundnut,NaN
1,Ofada Rice,₦701.00
2,Chicken Gizzard,"₦1,599.00"
3,Goat meat,"₦1,698.08"
4,Exeter Corned Beef 200g,₦555.55


In [94]:
items.shape

(321, 2)

In [54]:
items.isnull().any()

Name     False
Price    False
dtype: bool

In [95]:
#drop first row
items = items.iloc[1:]

In [36]:
items.head()

,Name,Price
1,Ofada Rice,₦701.00
2,Chicken Gizzard,"₦1,599.00"
3,Goat meat,"₦1,698.08"
4,Exeter Corned Beef 200g,₦555.55
5,Exeter Corned Beef 340g,₦855.55


In [55]:
items.shape

(321, 2)

In [56]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    321 non-null    object
 1   Price   321 non-null    object
dtypes: object(2)
memory usage: 5.1+ KB


In [60]:
items.tail()

,Name,Price
315,Chicken Gizzard,"₦1,599.00"
316,Ofada Rice,₦701.00
318,Garri (White),₦170.00
319,Peppersoup Spice,₦100.00
320,Honeywell Semolina,₦302.00


In [96]:
items = items[~items.Price.str.contains('NaN')]  #creates a variable items that does nt contain any row where price=NaN

In [62]:
items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 1 to 320
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    319 non-null    object
 1   Price   319 non-null    object
dtypes: object(2)
memory usage: 7.5+ KB


In [64]:
items.head()

,Name,Price
1,Ofada Rice,₦701.00
2,Chicken Gizzard,"₦1,599.00"
3,Goat meat,"₦1,698.08"
4,Exeter Corned Beef 200g,₦555.55
5,Exeter Corned Beef 340g,₦855.55


In [97]:
items["Price"] = items["Price"].str.replace('₦', '') 

In [66]:
items.head()

,Name,Price
1,Ofada Rice,701.00
2,Chicken Gizzard,"1,599.00"
3,Goat meat,"1,698.08"
4,Exeter Corned Beef 200g,555.55
5,Exeter Corned Beef 340g,855.55


In [98]:
items["Price"] = [x[:-3] for x in items["Price"]] #removes eerythig after . in price.last three characters

In [99]:
items["Price"].head()

1      701
2    1,599
3    1,698
4      555
5      855
Name: Price, dtype: object

In [100]:
items.to_csv("Ounje_Inventory.csv",index=False)